In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
1+1

2

In [4]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

In [7]:
len(docs)

18846

In [21]:
_docs = docs[:200]

# topic_model = BERTopic()
# topics, probs = topic_model.fit_transform(_docs)

In [22]:
# topic_model.get_topic_info()

In [23]:
from sentence_transformers import SentenceTransformer

# Pre-calculate embeddings
embedding_model = SentenceTransformer("BAAI/bge-small-en")
embeddings = embedding_model.encode(_docs, show_progress_bar=True)

Batches: 100%|██████████| 7/7 [00:02<00:00,  2.48it/s]


In [24]:
from umap import UMAP
from hdbscan import HDBSCAN

umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=150, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [ ]:
from package.custom_representation import CustomLLM
from package.llm import Ollama, Topic
import json

def parse_json(content):
    content = content.split("```json")[-1]
    content = content.split("```")[0]
    return Topic(**json.loads(content))

def Output(response):
    content = response['message']['content']
    return parse_json(content)

llm = Ollama(Output=Output)

representation_model = CustomLLM(llm=llm)

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(

  # Sub-models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  # representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

# Train model
topics, probs = topic_model.fit_transform(_docs, embeddings)

2025-12-15 23:07:35,562 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-12-15 23:07:35,772 - BERTopic - Dimensionality - Completed ✓
2025-12-15 23:07:35,773 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-12-15 23:07:35,779 - BERTopic - Cluster - Completed ✓
2025-12-15 23:07:35,780 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-12-15 23:07:35,794 - BERTopic - Representation - Completed ✓


In [ ]:
topic_model._extract_representative_docs()

In [26]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,200,-1_the_to_of_and,"[the, to, of, and, in, is, that, it, you, for]","[\n\nIf a 'dog's prayers were answered, bones ..."


In [ ]:
system_prompt = """
Always return in JSON codeblock as the following schema:
```json
{"topic": "extracted topic"}
```
""".strip()

llm.run(system_prompt, "USER_INPUT:\n\nI love pizza!")

Topic(topic='pizza')